## Part V: Training a Model with Data Programming

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ features snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`, feature matrix, and label matrix

In [4]:
from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [5]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

%time F_train = feature_manager.load(session, train, 'Train Features')

CPU times: user 18.6 s, sys: 676 ms, total: 19.3 s
Wall time: 19.4 s


In [6]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

%time L_train = label_manager.load(session, train, 'LF Labels')

CPU times: user 187 ms, sys: 15.1 ms, total: 202 ms
Wall time: 208 ms


## Train Generative Model

In [7]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=3000, rate=1e-5)

Training marginals (!= 0.5):	3630
Features:			12
Begin training for rate=1e-05, mu=1e-06
	Learning epoch = 0	Gradient mag. = 0.372587
	Learning epoch = 250	Gradient mag. = 0.392190
	Learning epoch = 500	Gradient mag. = 0.392183
	Learning epoch = 750	Gradient mag. = 0.392176
	Learning epoch = 1000	Gradient mag. = 0.392169
	Learning epoch = 1250	Gradient mag. = 0.392162
	Learning epoch = 1500	Gradient mag. = 0.392156
	Learning epoch = 1750	Gradient mag. = 0.392149
	Learning epoch = 2000	Gradient mag. = 0.392142
	Learning epoch = 2250	Gradient mag. = 0.392135
	Learning epoch = 2500	Gradient mag. = 0.392129
	Learning epoch = 2750	Gradient mag. = 0.392122
Final gradient magnitude for rate=1e-05, mu=1e-06: 0.392


In [8]:
gen_model.save(session, 'Generative Params')

In [9]:
train_marginals = gen_model.marginals(L_train)

## Train Discriminative Model

In [10]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=5000, rate=1e-3)

Training marginals (!= 0.5):	3515
Features:			25256
Using gradient descent...
	Learning epoch = 0	Step size = 0.001
	Loss = 2436.412340	Gradient magnitude = 5282.868771
	Learning epoch = 100	Step size = 0.000904792147114
	Loss = 6532.274418	Gradient magnitude = 1583.228523
	Learning epoch = 200	Step size = 0.000818648829479
	Loss = 3861.986866	Gradient magnitude = 1807.607434
	Learning epoch = 300	Step size = 0.000740707032156
	Loss = 4254.648488	Gradient magnitude = 4437.103422
	Learning epoch = 400	Step size = 0.000670185906007
	Loss = 9048.262055	Gradient magnitude = 3965.658307
	Learning epoch = 500	Step size = 0.000606378944861
	Loss = 3982.634282	Gradient magnitude = 2895.942124
	Learning epoch = 600	Step size = 0.000548646907485
	Loss = 5943.963233	Gradient magnitude = 1510.664241
	Learning epoch = 700	Step size = 0.000496411413431
	Loss = 3170.600541	Gradient magnitude = 4070.877473
	Learning epoch = 800	Step size = 0.00044914914861
	Loss = 3236.211738	Gradient magnitude = 3037

In [11]:
disc_model.w.shape

(25256,)

In [12]:
%time disc_model.save(session, "Discriminative Params")

CPU times: user 6.79 s, sys: 209 ms, total: 7 s
Wall time: 6.85 s


## Assess Performance on Development Set

In [13]:
from snorkel.models import CandidateSet
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()

### Loading the Development Feature Matrix

In [14]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()
%time F_dev = feature_manager.load(session, dev, 'Train Features')

CPU times: user 13.3 s, sys: 434 ms, total: 13.7 s
Wall time: 13.7 s


In [15]:
L_dev = label_manager.load(session, dev, "Hardware Development Labels -- Gold")

In [16]:
gold_dev_set = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates -- Gold').one()

In [17]:
F_dev.shape

(1292, 25256)

In [18]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)

Calibration plot:
Recall-corrected Noise-aware Model
Pos. class accuracy: 0.73665480427
Neg. class accuracy: 0.82393669634
Corpus Precision 0.538
Corpus Recall    0.737
Corpus F1        0.622
----------------------------------------
TP: 207 | FP: 178 | TN: 833 | FN: 74

Recall-corrected Noise-aware Model
Pos. class accuracy: 0.73665480427
Neg. class accuracy: 0.82393669634
Corpus Precision 0.538
Corpus Recall    0.737
Corpus F1        0.622
----------------------------------------
TP: 207 | FP: 178 | TN: 833 | FN: 74



/Users/bradenhancock/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [19]:
def error_analysis(c):
    print c[0]
    print c[0].parent
    print (c[0].parent.row_num, c[0].parent.col_num)
    print c[1]
    print c[1].parent
    print (c[1].parent.row_num, c[1].parent.col_num)

In [20]:
c = list(tp)[10]
error_analysis(c)
print c.temp.parent.html_anc_tags

Span("2N4124", parent=15175, chars=[0,5], words=[0,0])
Phrase(Document 2N4124, 545, None, 0, u'2N4124  /  MMBT4124')
(None, None)
Span("-55", parent=90797, chars=[0,2], words=[0,0])
Phrase(Document 2N4124, 544, 15137, 0, u'-55 to +150')
(5, 2)
['html', 'body', 'table', 'tr', 'td', 'p']


In [27]:
from snorkel.models import Corpus
from hardware_utils import entity_level_f1
import os

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
entity_level_f1(tp, fp, tn, fn, filename, corpus, 'stg_temp_min')

Scoring on Entity-Level Gold Data
Corpus Precision 0.627
Corpus Recall    0.598
Corpus F1        0.612
----------------------------------------
TP: 64 | FP: 38 | FN: 43



In [22]:
import os
os.system('cp snorkel.db snorkel.db\ final');

The End.